# Shadow Detection for Mobile Robots

In [ ]:
%pylab notebook
pylab.rcParams['figure.figsize'] = 10, 8

from IPython.display import YouTubeVideo as yt
def YouTubeVideo(_id):
    return yt(_id, width=800, height=450)

from skimage import io

from shadow import _imshow

## Recap of Project

This project is about *shadow detection for mobile robots* - detecting shadows with an *active camera*. This is challenging because the majority of previous work revolves around using a background model to detect changed parts of a scene. This obviously can't work for active video!

### Hypothesis

The detection of shadows is possible by combining information about texture features (something that remains largely unchanged under shadow) with other information - including local maxima, smoothness, or other colourimetric features.

### Key Points

* Capturing multiple datasets
* Simple shadow detection with colour/brightness
* Texture feature investigation
* Clustering texture features into contiguous regions
* Combining 
* Machine learning techniques

## Datasets

### Static Camera (or "easy")

In [ ]:
images = [imread("../data/static/%s/images/small-00044.png" % (i))
          for i in ["bobbly-slabs", "bricks", "smooth-slabs", "tarmac"]]  
_imshow(2, False, *images)

### Active Camera (or "slightly less easy")

In [ ]:
images = [imread("../data/active/%s/images/small-00044.png" % (i))
          for i in ["grass-path", "seafront-gravel", "seafront-path"]]  
_imshow(2, False, *images)

### Ground Truth

The following video is an example of some of the initial techniques that I was using to generate ground truth. This involved using local brightness maxima and some morphological post-processing to reduce noise and fill holes (opening/closing). This is pretty much the point I'd reached at the time of the mid-project demonstration.

In [ ]:
YouTubeVideo("j5PmcK6b4dQ")

In [ ]:
# So, for example, the ground truth looks like this...
image = imread("../data/static/smooth-slabs/images/small-00052.png")
ground_truth = imread("../data/static/smooth-slabs//ground-truth/small-00052.png")
_imshow(1, False, image, ground_truth)

## Texture

### Choice of Texture Feature

A large part of this project revolved around choosing texture features based on their shadow invariance. In the end, local binary patterns turned out to be the *best* texture feature - although, LBPs were chosen based on a combination of factors, rather than solely shadow invariance.

### Flaws in Analytical Method

The simplest way of comparing the shadow invariance of texture features seemed to be extracting two sets of features for an image pair - one image under shadow, and the other with no shadow - then using a simple metric for difference, such as mean squared error (essentially, Euclidean distance).

However, as became clear during experimentation, different feature types can't necessarily be compared in this manner as their data structures can be completely different. A sparse data structure, (for example, from a GLCM - a mostly-empty array), may exhibit a *very low* mean squared error - whereas a more dense structure (for example, an LBP) may give a considerably different result.

I would have changed this or investigated this more, given time.

### Texture Feature Clustering

K-means was the primary clustering technique used in this project. As well as having a particularly performant mini-batch implementation in `scikit-learn`, it was also the most straightforward algorithm to use - its only drawback was having to specify $k$.

In [ ]:
import shadow

image = io.imread("../data/static/bricks/images/small-00015.png", as_grey=True)
clusters = shadow.k_means_texture_cluster(image)

_imshow(2, False, image, clusters)

#### Notable Mention

It's annoying to have to specify $k$ for each image, particularly on unknown image sequences. Online, unsupervised estimation of $k$ is a whole other project in itself, so I also tried DBSCAN for clustering (which is a clustering algorithm that does not have this requirement). DBSCAN essentially looks for areas of density separated by sparse areas. It was incredibly slow (almost unusable, actually), so it wasn't used in the end - but it was worth mentioning.

### Testing Clustering Quality

Because of the flawed analysis of shadow-invariant texture features, instead I evaluated how consistent texture clusters were under shadow. This simply involved applying a clustering algorithm to texture features from shadow-free/shadowed image pairs, and comparing the resultant clusters with various metrics (e.g. Adjusted Rand Index, Homogeneity, etc).

## Simple Clustering-Based Shadow Detection

### Without Variance Threshold

Note that a considerable proportion of the input image is detected as shadow. These false positives are created due to considerable variations in brightness at a particularly fine scale - caused by the small gravel particles.

In [ ]:
import shadow

image = io.imread("../data/static/tarmac/images/small-00170.png", as_grey=True)
ground_truth = io.imread("../data/static/tarmac/ground-truth/small-00170.png", as_grey=True)
shadow_pred = shadow.k_means_classifier(image)
_imshow(1, False, image, shadow_pred, ground_truth)

### With Variance Threshold

Note that the variance threshold suppresses all of the superfluous shadow detection.

In [ ]:
import shadow

image = io.imread("../data/static/tarmac/images/small-00170.png", as_grey=True)
ground_truth = io.imread("../data/static/tarmac/ground-truth/small-00170.png", as_grey=True)
shadow_pred = shadow.k_means_variance_classifier(image)
_imshow(1, True, image, shadow_pred, ground_truth)

## Machine Learning

Very close to the end of the project, there were one or two days spare - so it made sense to experiment with some machine learning techniques before time ran out. Two types were trialled - decision trees and random forests.

### Decision Trees

Decision trees were the most effective machine learning technique used in this project, mostly due to their extreme simplicity and excellent performance.

In [ ]:
YouTubeVideo("zBXaazf0vI8")

### Random Forests

Random forests yielded very similar output to decision trees, with a little less overfitting. They were a little more effective than decision trees (of the order of a few percent), but were incredibly resource-intensive - so were somewhat impractical. Nonetheless, here's a quick sample of some output.

In [ ]:
YouTubeVideo("GQUTqY1oths")